In [2]:
import nb_setup 
import importlib
importlib.reload(nb_setup)
nb_setup.init()

Changed current working directory to: /Users/falcon/Developments/act/backtesting-playground/trading
Initialized project with base directory: /Users/falcon/Developments/act/backtesting-playground/trading


In [3]:
import numpy as np
import pandas as pd

def rank_column(df, column, ascending=False):
    _df = df.copy()
    col_name = f"rank_{column}"
    if col_name not in _df.columns:
        _df[col_name] = np.nan
    _df.loc[_df[col_name].isna(), col_name] = _df.groupby("start_time")[column].rank(
        ascending=ascending
    )
    return _df    

def prepare_data(data: pd.DataFrame):
    """
    Prepare the data for the strategy
    """
    data = data.copy()
    window_size = 7 * 24 * 4  # Adjust this based on your exact data frequency
    prep_col = [
        "pct_change",
        "rolling_accumulated_pct_change",
        "rolling_variance_pct_change",
        "rank",
    ]
    for col in prep_col:
        if col not in data.columns:
            data[col] = np.nan
    ### pct chage
    data.loc[data["pct_change"].isna(), "pct_change"] = data.groupby(level=0)[
        "close"
    ].pct_change()
    ### rolling_acc_pct_change
    data.loc[
        data["rolling_accumulated_pct_change"].isna(),
        "rolling_accumulated_pct_change",
    ] = data.groupby(level=0)["pct_change"].transform(
        lambda x: (
            x.rolling(window=window_size, min_periods=1).apply(
                lambda y: np.prod(1 + y / 100)
            )
            - 1
        )
        * 100
    )
    data.loc[
        data["rolling_variance_pct_change"].isna(), "rolling_variance_pct_change"
    ] = data.groupby(level=0)["pct_change"].transform(
        lambda x: x.rolling(window=window_size, min_periods=1).var()
    )
    data = rank_column(data, "rolling_accumulated_pct_change", ascending=False)
    data = rank_column(data, "rolling_variance_pct_change", ascending=True)
    data.loc[data["rank"].isna(), "rank"] = (
        data["rank_rolling_accumulated_pct_change"]
        + data["rank_rolling_variance_pct_change"]
    ) / 2
    return data

In [4]:
from settings import DATA_DIR

df = pd.read_csv(DATA_DIR / "all.csv", parse_dates=[1, 2])
df.set_index(["symbol", "start_time"], inplace=True)
ranked_df = prepare_data(df)

In [6]:
data = rank_column(ranked_df, "rolling_accumulated_pct_change", ascending=False)

In [7]:
data.head()

end_time interval  number_of_trades  \
symbol start_time                                                           
SOL    2024-03-09 00:00:00 2024-03-09 00:14:59      15m               436   
       2024-03-09 00:15:00 2024-03-09 00:29:59      15m               399   
       2024-03-09 00:30:00 2024-03-09 00:44:59      15m               402   
       2024-03-09 00:45:00 2024-03-09 00:59:59      15m               286   
       2024-03-09 01:00:00 2024-03-09 01:14:59      15m               421   

                             close    high     low    open    volume  \
symbol start_time                                                      
SOL    2024-03-09 00:00:00  146.12  146.39  144.84  145.09   9451.39   
       2024-03-09 00:15:00  146.24  146.31  145.72  146.10   8242.40   
       2024-03-09 00:30:00  146.54  146.59  145.18  146.20   8876.19   
       2024-03-09 00:45:00  146.40  147.20  146.27  146.50   7156.97   
       2024-03-09 01:00:00  144.95  146.42  144.95  146.41  24934.58   

                                      tic            toc  pct_change  \
symbol start_time                                                      
SOL    2024-03-09 00:00:00  1709942400000  1709943299999         NaN   
       2024-03-09 00:15:00  1709943300000  1709944199999    0.000821   
       2024-03-09 00:30:00  1709944200000  1709945099999    0.002051   
       2024-03-09 00:45:00  1709945100000  1709945999999   -0.000955   
       2024-03-09 01:00:00  1709946000000  1709946899999   -0.009904   

                            rolling_accumulated_pct_change  \
symbol start_time                                            
SOL    2024-03-09 00:00:00                             NaN   
       2024-03-09 00:15:00                        0.000821   
       2024-03-09 00:30:00                        0.002873   
       2024-03-09 00:45:00                        0.001917   
       2024-03-09 01:00:00                       -0.007987   

                            rolling_variance_pct_change  rank  \
symbol start_time                                               
SOL    2024-03-09 00:00:00                          NaN   NaN   
       2024-03-09 00:15:00                          NaN   NaN   
       2024-03-09 00:30:00                 7.566708e-07   1.0   
       2024-03-09 00:45:00                 2.285083e-06   1.0   
       2024-03-09 01:00:00                 2.931458e-05   1.0   

                            rank_rolling_accumulated_pct_change  \
symbol start_time                                                 
SOL    2024-03-09 00:00:00                                  NaN   
       2024-03-09 00:15:00                                  1.0   
       2024-03-09 00:30:00                                  1.0   
       2024-03-09 00:45:00                                  1.0   
       2024-03-09 01:00:00                                  1.0   

                            rank_rolling_variance_pct_change  
symbol start_time                                             
SOL    2024-03-09 00:00:00                               NaN  
       2024-03-09 00:15:00                               NaN  
       2024-03-09 00:30:00                               1.0  
       2024-03-09 00:45:00                               1.0  
       2024-03-09 01:00:00                               1.0

In [5]:
ranked_df.head()

end_time interval  number_of_trades  \
symbol start_time                                                           
SOL    2024-03-09 00:00:00 2024-03-09 00:14:59      15m               436   
       2024-03-09 00:15:00 2024-03-09 00:29:59      15m               399   
       2024-03-09 00:30:00 2024-03-09 00:44:59      15m               402   
       2024-03-09 00:45:00 2024-03-09 00:59:59      15m               286   
       2024-03-09 01:00:00 2024-03-09 01:14:59      15m               421   

                             close    high     low    open    volume  \
symbol start_time                                                      
SOL    2024-03-09 00:00:00  146.12  146.39  144.84  145.09   9451.39   
       2024-03-09 00:15:00  146.24  146.31  145.72  146.10   8242.40   
       2024-03-09 00:30:00  146.54  146.59  145.18  146.20   8876.19   
       2024-03-09 00:45:00  146.40  147.20  146.27  146.50   7156.97   
       2024-03-09 01:00:00  144.95  146.42  144.95  146.41  24934.58   

                                      tic            toc  pct_change  \
symbol start_time                                                      
SOL    2024-03-09 00:00:00  1709942400000  1709943299999         NaN   
       2024-03-09 00:15:00  1709943300000  1709944199999    0.000821   
       2024-03-09 00:30:00  1709944200000  1709945099999    0.002051   
       2024-03-09 00:45:00  1709945100000  1709945999999   -0.000955   
       2024-03-09 01:00:00  1709946000000  1709946899999   -0.009904   

                            rolling_accumulated_pct_change  \
symbol start_time                                            
SOL    2024-03-09 00:00:00                             NaN   
       2024-03-09 00:15:00                        0.000821   
       2024-03-09 00:30:00                        0.002873   
       2024-03-09 00:45:00                        0.001917   
       2024-03-09 01:00:00                       -0.007987   

                            rolling_variance_pct_change  rank  \
symbol start_time                                               
SOL    2024-03-09 00:00:00                          NaN   NaN   
       2024-03-09 00:15:00                          NaN   NaN   
       2024-03-09 00:30:00                 7.566708e-07   1.0   
       2024-03-09 00:45:00                 2.285083e-06   1.0   
       2024-03-09 01:00:00                 2.931458e-05   1.0   

                            rank_rolling_accumulated_pct_change  \
symbol start_time                                                 
SOL    2024-03-09 00:00:00                                  NaN   
       2024-03-09 00:15:00                                  1.0   
       2024-03-09 00:30:00                                  1.0   
       2024-03-09 00:45:00                                  1.0   
       2024-03-09 01:00:00                                  1.0   

                            rank_rolling_variance_pct_change  
symbol start_time                                             
SOL    2024-03-09 00:00:00                               NaN  
       2024-03-09 00:15:00                               NaN  
       2024-03-09 00:30:00                               1.0  
       2024-03-09 00:45:00                               1.0  
       2024-03-09 01:00:00                               1.0